In [ ]:
#imports

%load_ext autoreload

%autoreload 2

import sys
sys.path.insert(0, '/home/users/aadams0/microwell_code/processingpack')

from copy import deepcopy
from pathlib import Path

import numpy as np
import skimage
from skimage import measure
import pandas as pd
from tqdm import tqdm
%matplotlib inline
import matplotlib.pyplot as pl

from skimage import data, filters, io
import operator

from mpl_toolkits.axes_grid1 import make_axes_locatable
import os
import cv2 as cv

import ast
import operator

import chip
import experiment as exp
import chipcollections as collections
from org_extension import *


%config InlineBackend.figure_format = 'svg'

In [ ]:
#visualize wells - start with last image in the timecourse (will repeat for earliest image after doing the next 2 steps)

mCherry_imagePath = '/scratch/users/aadams0/processed_data/Nikon/4230_P53_ARID1A_200um_20200710/B3/mCherry/timepoint_20200710_193417-7-20201124.tif'

with skimage.external.tifffile.TiffFile(mCherry_imagePath) as mCherry_tif:
    img = mCherry_tif.asarray()
        
# if not ax:
#     ax = pl.gca()
    
fig, ax = pl.subplots(figsize=(8.5,11))

im = ax.imshow(img[0])
cbar = ax.figure.colorbar(im, ax=ax)


In [ ]:
#plot pixel intensity distribution

fig = pl.figure(figsize=(6,3))
ax = fig.add_subplot(111)
ax.hist(img[0].ravel(), bins = 100, range = (0,1000))
ax.set_yscale('log')

In [ ]:
#pick threshold separating peak and tail of distribution above and test outcome
#adjust threshold up and down until there is no background coming through but all cells seen in image above are visible
#then repeat the last 3 steps with the earliest image in the timelapse to confirm the same threshold works

thresh = 300
ret,th1 = cv.threshold(img[0],thresh,65535,cv.THRESH_BINARY)

fig, ax = pl.subplots(figsize=(8.5,11))

im = ax.imshow(th1)
cbar = ax.figure.colorbar(im, ax=ax)

print(np.sum(th1)/65535)

In [ ]:
#save tif images of individual microwells and csv with organoid area per microwell

#path to folder where you saved timelapse images from timecourse processing 
input_folder = '/scratch/users/aadams0/processed_data/Nikon/4230_P53_ARID1A_200um_20200710/B3/mCherry/'
#change to reflect well being processed (A1 = 1, B1= 5 etc)
well_number = 7
#change to folder where you want to save csv
output_csv = '/scratch/users/aadams0/multi_plot_well_data2/indiv_csv/4230_P53_ARID1A_200um_20200710/B3/well_summary_B3indiv.csv'
#change to threshold determined aboe
thresh = 300

with open(output_csv, "w") as out:
    header = 'well id' + ',' + 'timepoint' + ',' + 'median pixel intensity' + ',' +'min pixel intensity' + ',' + 'max pixel intensity' + ',' + 'mCherry area' + "\n"
    out.write(header)

#iterate through all days in timecourse - same list as used in timecourse processing, but with "_" instead of "-"    
for timepoint in ('20200710_193417', '20200710_214615', '20200710_235759', '20200711_020946', '20200711_042130', '20200711_063307', '20200711_084438', '20200711_105613', '20200711_130757', '20200711_151939', '20200711_173116', '20200711_194255', '20200711_215426', '20200712_000601', '20200712_021740', '20200712_042917', '20200712_064040', '20200712_085211', '20200712_110344', '20200712_131518', '20200712_152650', '20200712_173822', '20200712_194954', '20200712_220127', '20200713_001256', '20200713_022427', '20200713_043558', '20200713_064726', '20200713_085859', '20200713_111027', '20200713_132201', '20200713_153321', '20200713_174450', '20200713_195620', '20200713_220751', '20200714_001920', '20200714_023042', '20200714_044209', '20200714_065335', '20200714_090459', '20200714_111626', '20200714_132754', '20200714_153926', '20200714_175055', '20200714_200227', '20200714_221358', '20200715_002530', '20200715_023703', '20200715_044836', '20200715_070008', '20200715_091138', '20200715_112302', '20200715_133322', '20200715_154341', '20200715_175507', '20200715_200643', '20200715_221819', '20200716_002953', '20200716_024127', '20200716_045305', '20200716_070431', '20200716_091544'):
    #change date to what you used for timecourse processing
    mCherry_reimport = pd.read_csv(output_folder + 'timepoint_{0}-{1}-20201124.csv'.format(timepoint,well_number))
    mCherry_imagePath = output_folder + 'timepoint_{0}-{1}-20201124.tif'.format(timepoint,well_number)
    with skimage.external.tifffile.TiffFile(mCherry_imagePath) as mCherry_tif:
        mCherry_data = mCherry_tif.asarray()
        
    #find inside of microwell area for first n wells
    for i in range(len(mCherry_reimport)):
        chamberInfo = mCherry_reimport.iloc[i]
        xslice = ast.literal_eval(chamberInfo.summaryImg_xslice)
        yslice = ast.literal_eval(chamberInfo.summaryImg_yslice)
        mCherry_well = mCherry_data[chamberInfo.stack_indexer][xslice[0]:xslice[1],yslice[0]:yslice[1]]
        #change to folder where you will save microwell images
        cv.imwrite('/scratch/users/aadams0/multi_plot_well_data2/indiv_tifs/4230_P53_ARID1A_200um_20200710/B3_P53_ARID1A/well{0:04d}_timepoint{1}_mCherry_well.tif'.format(i, timepoint), mCherry_well)
                
#         blur = cv.medianBlur(mCherry_uint8,5,0)

#         mCherry_blur = cv.blur(mCherry_well, (5,5))
        ret,mCherry_th1 = cv.threshold(mCherry_well,thresh,65535,cv.THRESH_BINARY)
#             cv.imwrite(output_folder + 'vegan3/well{0:04d}_timepoint{1}_mCherry_th1.tif'.format(i, timepoint), mCherry_th1)
        
        mCherry_area = (np.sum(mCherry_th1)/65535)
        
        with open(output_csv, "a") as output:
            line = str(i) + ',' + str(timepoint) + ',' + str(np.median(mCherry_well)) + ',' + str(np.min(mCherry_well)) + ',' + str(np.max(mCherry_well)) + ',' + str(mCherry_area) +"\n"
            output.write(line)
        
        print('well' + " " +  str(i) + " " + 'timepoint' + " " + str(timepoint))